In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Prepare data

In [ ]:
df = pd.read_csv('merged.csv')

X = df.drop(['label'], axis=1)  # Drop 'label' column
y = df['label'] # Keep only labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


Prepare data with specified ratios of malicious/benign data

In [ ]:
df = pd.read_csv('merged.csv')

# Split the dataset into malicious and benign subsets
malicious_df = df[df['label'] == 1]
benign_df = df[df['label'] == 0]

# Desired test size and malicious ratio in test set
test_size = 0.2 
malicious_ratio_in_test = 0.1

# Calculate the number of malicious samples in the test set
total_test_samples = int(len(df) * test_size)
malicious_test_samples = int(total_test_samples * malicious_ratio_in_test)
benign_test_samples = total_test_samples - malicious_test_samples

# Split the malicious data
malicious_train, malicious_test = train_test_split(malicious_df, test_size=malicious_test_samples, random_state=42)

# Split the benign data
benign_train, benign_test = train_test_split(benign_df, test_size=benign_test_samples, random_state=42)

# Combine the training and testing sets
train_df = pd.concat([malicious_train, benign_train])
test_df = pd.concat([malicious_test, benign_test])

# Shuffle the training and testing sets to ensure random distribution
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and labels for training and testing sets
X_train = train_df.drop(['label'], axis=1)
y_train = train_df['label']
X_test = test_df.drop(['label'], axis=1)
y_test = test_df['label']

# Print the sizes of the training and test data
print(f'Training set size: {len(X_train)} samples')
print(f'Test set size: {len(X_test)} samples')

# Output the shapes to verify the splits
print(f'Training set shape: {X_train.shape}, {y_train.shape}')
print(f'Test set shape: {X_test.shape}, {y_test.shape}')
print(f'Class distribution in test set:\n{y_test.value_counts()}')

Train RF Classifier

In [ ]:
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

Make predictions

In [ ]:
# Predict on the test set
y_pred = rf_model.predict(X_test)

Evaluate results

In [ ]:
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

Examine feature importance

In [ ]:
# Feature importance
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
print('Feature Importances:')
print(feature_importances)

Examine results

In [ ]:
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Display the results
print(results_df)